In [34]:
import sys
from Preprocess.SentTokenizer import SentTokenizer
import pandas as pd
from collections import defaultdict
import tensorflow as tf
from estimators import ElmoModel
import json
from brat_parser import get_entities_relations_attributes_groups

# converter BIO -> ANN

In [25]:
type_d = {
    'B_ORG':'Company',
    'B_PER':'Person',
    'B_MEDIA':'Media',
    'B_SPORT':'Sport',
    'B_LOC':'Location'
}


def get_span(text_splt, id_):
    lens = 0
    ids_ = id_
    while ids_ >= 0:
        lens += len(text_splt[ids_])
        ids_ -= 1
    lens += id_
    return lens-len(text_splt[id_]), lens

def from_BIO_to_ANN(bio, text, text_splt):

    d_ = defaultdict(dict)
    ann_d = defaultdict(dict)
    
    for id_ , tag in enumerate(bio):
        span_start = None
        d_[id_] = {
                    'word':text_splt[id_], 
                    'span_start': get_span(text_splt, id_)[0],
                    'span_end': get_span(text_splt, id_)[1],
                    'tag':tag
                  }

    for k, v in d_.items():
        if v['tag'].startswith('B_'):
            how_to_go_back = 0
            ann_T = 'T'+str(k)
            ann_d[ann_T] = {
                'type': type_d[v['tag']],
                'span_start': v['span_start'],
                'span_end': v['span_end'],
            }
        if v['tag'].startswith('I_'):
            how_to_go_back += 1
            ann_T = 'T'+str(k-how_to_go_back)
            ann_d[ann_T]['span_end'] = v['span_end']

    for k in ann_d.keys():
        ann_d[k]['words'] = text[ann_d[k]['span_start']:ann_d[k]['span_end']]
        
    return ann_d

def ann_to_string(ann_d):
    # {'T6': {'type': 'Media',
    #               'span_start': 33,
    #               'span_end': 51,
    #               'words': '" Время новостей "'},
    #   ->
    # T6 Media 33 51 " Время новостей "
    
    string = ''
    for k , v in ann_d.items():
        string += k + '	'
        string += v['type'] + ' '
        string += str(v['span_start']) + ' '
        string += str(v['span_end']) + '	'
        string += v['words'] + '\n'
    return string    

In [340]:
snts_error = sentences[542]
print(snts_error)
model.predict_fast([snts_error])

Ожидается, что на Итурупе Медведев посетит ряд социальных и инфраструктурных объектов.


[['O', 'O', 'O', 'O', 'B_LOC', 'B_PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]

In [342]:
# bio_str = 'O O O O O O B_MEDIA I_MEDIA I_MEDIA I_MEDIA O B_SPORT B_LOC B_PER I_PER O'
bio = ['O', 'O', 'O', 'O', 'B_LOC', 'B_PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
text = 'Ожидается , что на Итурупе Медведев посетит ряд социальных и инфраструктурных объектов .'
text_splt = text.split()


ann_d = from_BIO_to_ANN(bio, text, text_splt)
string_wrt = ann_to_string(ann_d)
with open("made_ANN_TXT/0.ann", "w") as text_file:
    text_file.write(string_wrt)
with open("made_ANN_TXT/0.txt", "w") as text_file:
    text_file.write(text)    
ann_d    

defaultdict(dict,
            {'T4': {'type': 'Location',
              'span_start': 19,
              'span_end': 26,
              'words': 'Итурупе'},
             'T5': {'type': 'Person',
              'span_start': 27,
              'span_end': 35,
              'words': 'Медведев'}})

### конвертор ANN -> BIO

In [326]:
type_ann = {
    'Company':'ORG',
    'Person':'PER',
    'Media':'MEDIA',
    'Sport':'SPORT',
    'Location':'LOC'
}

def from_ANN_to_BIO(entities, lines):

    txt = lines[0]
    ann_dict = defaultdict(dict)
    position_ann_dict = defaultdict(dict)

    for k, v in entities.items():
        ann_dict[v.span[0]] = {
            'type':v.type,
            'text':v.text,
            'T': k
        }


    for k, v in ann_dict.items():
        start = k[0]
        aa = v['text'].split()
        for a in aa:
            position_a = start + len(a)
            position_ann_dict[(start,position_a)]={
                'text': txt[start:position_a],
                'type': v['type'],
                'T':v['T']
            }
            start = position_a + 1


    start = 0
    end = 0
    brat_bio_list = []
    last_T = None

    
    for pst, v in enumerate(txt.split()):
        if pst > 0:
            end += 1
        end += len(v)
 
        if position_ann_dict[(start, end)]:
            T = position_ann_dict[(start, end)]['T']
            if last_T is not None and last_T == T:
                brat_bio_list.append('I_' + type_ann[position_ann_dict[(start, end)]['type']])
            else:
                brat_bio_list.append('B_' + type_ann[position_ann_dict[(start, end)]['type']])
                last_T = position_ann_dict[(start, end)]['T']
            last_T = position_ann_dict[(start, end)]['T']                
        else:
            brat_bio_list.append('O')
            last_T = None
        start = end  + 1

    return brat_bio_list        

In [ ]:
file_name = '2'
entities, _, _, _ = get_entities_relations_attributes_groups(f"made_ANN_TXT/{file_name}.ann")
with open(f"made_ANN_TXT/{file_name}.txt", "r") as f:
    lines = f.readlines()

from_ANN_to_BIO(entities, lines)

### Test bio -> ann -> bio 

In [329]:
def test_bio_ann_bio(bio_1, bio_2, text_splt):
    for b1, b2 in zip(bio_1, bio_2):
        if b1 != b2:
            # return f"b1 {b1} \n b2 {b2} \n bio_1 {bio_1}\n bio_2 {bio_2}\n text_splt {text_splt}\n"
            return False
    return True

### Load NER model

In [ ]:
tf.reset_default_graph()
sess = tf.Session()
model = ElmoModel(sess)
head_dir = "./NER_MODEL/"
model.restore(head_dir)

### Prepare data for NER

In [ ]:
def load_data(path):
    with open(path) as f:
        sentences = json.load(f)
    return sentences

MIN_LEN = 30
MAX_LEN = 400

sentences = load_data('data_1m_sentences.json')
print(f'было {len(sentences)}')
sentences = [x for x in sentences if MIN_LEN < len(str(x)) < MAX_LEN]
print(f'стало {len(sentences)}')

In [349]:
preds = model.predict_fast(snts)
for e, p in enumerate(preds):
    if e not in bad_snts:
        try:        
            text_splt = model._tokenize(snts[e])
            text_after_tknz = ' '.join(text_splt)
            ann_d = from_BIO_to_ANN(p, text_after_tknz, text_splt)
            string_wrt = ann_to_string(ann_d)

            with open(f"made_ANN_TXT/{e}.ann", "w") as text_file:
                text_file.write(string_wrt)

            with open(f"made_ANN_TXT/{e}.txt", "w") as text_file:
                text_file.write(text_after_tknz)        

            with open(f"made_ANN_TXT/{e}.txt", "r") as f:
                lines = f.readlines()  

            entities, _, _, _ = get_entities_relations_attributes_groups(f"made_ANN_TXT/{e}.ann")
            bio_1 = from_ANN_to_BIO(entities, lines)  
            assert len(bio_1) == len(p), "Размер листов не совпадает"
            assert test_bio_ann_bio(bio_1, p, text_splt), "Ошибка в создании BIO->ANN->BIO"
        except Exception as ex:
            snts_error = snts[e]
            print(f'#{e}\n')
            print(f'Exception {ex}\n')
            print(f'snts[e] {snts_error}\n')                
            print(f'string_wrt {string_wrt}\n')
            print(f'bio_1 {bio_1}\n')        
            print(f'p {p}\n')             